In [7]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127694")
exp = Experiment(workspace=ws, name="quick-starts-ws-127694")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-127694
Azure region: southcentralus
Subscription id: b4a122b5-b4d5-40e7-9878-57b87adf4a8b
Resource group: aml-quickstarts-127694


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name="aml-compute"

try:

    new_cluster=ComputeTarget(workspace=ws, name="aml-compute")
    print("Existing cluster detected")

except ComputeTargetException:
    print("New compute target creation in progress...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    new_cluster = ComputeTarget.create(ws, "new_cluster", compute_config)    
    new_cluster.wait_for_completion(show_output=True, max_nodes=4)         
    print("Cluster is ready")                                                             

Existing cluster detected


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
parameter_sampling = RandomParameterSampling( {
                        "--C":uniform(0.04,1),
                        "--max_iter":choice(10,20,30,40,50,60,70,80,90,100) 
}) 

# Specify a Policy
early_termination_policy = BanditPolicy (evaluation_interval=2,slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#import packages
from azureml.core import ScriptRunConfig
from azureml.core import Environment
script='train.py'
est=SKLearn(source_directory='.',entry_script='train.py',compute_target=new_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                                      hyperparameter_sampling=ps,
                                      primary_metric_name='Accuracy',
                                      primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                      policy=early_termination_policy,
                                      max_total_runs=4,
                                      max_concurrent_runs=4)

In [23]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run=exp.submit(hyperdrive_config,show_output=True )
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

NameError: name 'hyperdrive_config' is not defined

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
src= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimited_files(path=src)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
#splitting the dataset
from sklearn.model_selection import train_test_split
import pandas as pd

x_train, x_test, y_train, y_test=train_test_split(x, y, train_size=0.8, test_size=0.2, random_size=42)
training_set=pd.concat((x_train,y_train),axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=training_set,
    label_column_name='y',
    n_cross_validations=2,
    compute_target=old_cluster,
    debug-log='automl_errors',
    max_concurrent_iterations=4)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

NameError: name 'automl_config' is not defined

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###